In [1]:
import sys
# sys.path.append('../')
%reload_ext autoreload
%autoreload 2

import os 
import torch
import numpy as np

from torch.utils.tensorboard import SummaryWriter
from os.path import join as pjoin
from torch.distributions import Categorical
import json
import clip

import options.option_transformer as option_trans
import models.vqvae as vqvae
import utils.utils_model as utils_model
import utils.eval_trans as eval_trans
from dataset import dataset_TM_train
from dataset import dataset_TM_eval
from dataset import dataset_tokenize
import models.t2m_trans as trans
from options.get_eval_option import get_opt
from models.evaluator_wrapper import EvaluatorModelWrapper
import warnings
warnings.filterwarnings('ignore')

In [2]:
class Temp:
    def __init__(self):
        print('mock:: opt')
args = Temp()
args.dataname = args.dataset_name = 'kit'
args.down_t = 2

train_loader_token = dataset_tokenize.DATALoader(args.dataname, 1, unit_length=2**args.down_t)

from utils.word_vectorizer import WordVectorizer
w_vectorizer = WordVectorizer('./glove', 'our_vab')
val_loader = dataset_TM_eval.DATALoader(args.dataname, False, 32, w_vectorizer)

dataset_opt_path = 'checkpoints/kit/Comp_v6_KLD005/opt.txt' if args.dataname == 'kit' else 'checkpoints/t2m/Comp_v6_KLD005/opt.txt'

wrapper_opt = get_opt(dataset_opt_path, torch.device('cuda'))
eval_wrapper = EvaluatorModelWrapper(wrapper_opt)

mock:: opt


100%|██████████| 300/300 [00:00<00:00, 2869.13it/s]


Pointer Pointing at 0
Reading checkpoints/kit/Comp_v6_KLD005/opt.txt
Loading Evaluation Model Wrapper (Epoch 30) Completed!!


In [3]:
##### ---- Network ---- #####
clip_model, clip_preprocess = clip.load("ViT-B/32", device=torch.device('cuda'), jit=False)  # Must set jit=False for training
clip.model.convert_weights(clip_model)  # Actually this line is unnecessary since clip by default already on float16
clip_model.eval()
for p in clip_model.parameters():
    p.requires_grad = False

In [4]:
args.nb_code = 512 # 8192 # 
args.code_dim = 512 # 32 # 
args.output_emb_width = 512
args.down_t = 2
args.stride_t = 2
args.width = 512
args.depth = 3
args.dilation_growth_rate = 3
args.vq_act = 'relu'
args.vq_norm = None
args.quantizer = 'ema_reset'
args.mu = 0.99

net = vqvae.HumanVQVAE(args, ## use args to define different parameters in different quantizers
                       args.nb_code,
                       args.code_dim,
                       args.output_emb_width,
                       args.down_t,
                       args.stride_t,
                       args.width,
                       args.depth,
                       args.dilation_growth_rate)
args.embed_dim_gpt = 1024
args.clip_dim = 512
args.block_size = 51
args.num_layers = 9
args.n_head_gpt = 16
args.drop_out_rate = 0.1
args.ff_rate = 4
trans_encoder = trans.Text2Motion_Transformer(num_vq=args.nb_code, 
                                            embed_dim=args.embed_dim_gpt, 
                                            clip_dim=args.clip_dim, 
                                            block_size=args.block_size, 
                                            num_layers=args.num_layers, 
                                            n_head=args.n_head_gpt, 
                                            drop_out_rate=args.drop_out_rate, 
                                            fc_rate=args.ff_rate)
net.eval()
net.cuda()
trans_encoder.train()
trans_encoder.cuda()
''

''

In [5]:
count = 0
for batch in train_loader_token:
    pose, name = batch
    bs, seq = pose.shape[0], pose.shape[1]

    pose = pose.cuda().float() # bs, nb_joints, joints_dim, seq_len
    target = net.encode(pose)
    target = target.cpu().numpy()
    print('pose:', pose.shape, name)
    count += 1
    if count == 10:
        break

pose: torch.Size([1, 64, 251]) ('M00099',)
pose: torch.Size([1, 160, 251]) ('03902',)
pose: torch.Size([1, 84, 251]) ('M03783',)
pose: torch.Size([1, 52, 251]) ('00913',)
pose: torch.Size([1, 36, 251]) ('M00447',)
pose: torch.Size([1, 64, 251]) ('02120',)
pose: torch.Size([1, 80, 251]) ('02096',)
pose: torch.Size([1, 52, 251]) ('M00984',)
pose: torch.Size([1, 44, 251]) ('01254',)
pose: torch.Size([1, 40, 251]) ('01839',)


### VQVAE encoder index precomputed

In [6]:
train_loader_token = dataset_tokenize.DATALoader(args.dataname, 1, unit_length=2**args.down_t)

100%|██████████| 4888/4888 [00:01<00:00, 2713.84it/s]


In [7]:
for batch in train_loader_token:
    pose, name = batch
    bs, seq = pose.shape[0], pose.shape[1]

    pose = pose.cuda().float() # bs, nb_joints, joints_dim, seq_len
    target = net.encode(pose)
    target = target.cpu().numpy()
    break

### Train

In [8]:
args.batch_size = 2
args.vq_name = 'VQVAE'
train_loader = dataset_TM_train.DATALoader(args.dataname, args.batch_size, args.nb_code, args.vq_name, unit_length=2**args.down_t)
train_loader_iter = dataset_TM_train.cycle(train_loader)

100%|██████████| 4888/4888 [00:01<00:00, 4241.61it/s]


In [9]:
args.pkeep = .5
batch = next(train_loader_iter)
clip_text, m_tokens, m_tokens_len = batch
m_tokens, m_tokens_len = m_tokens.cuda(), m_tokens_len.cuda()
bs = m_tokens.shape[0]
target = m_tokens    # (bs, 26)
target = target.cuda()

text = clip.tokenize(clip_text, truncate=True).cuda()
    
feat_clip_text = clip_model.encode_text(text).float()

input_index = target[:,:-1]

if args.pkeep == -1:
    proba = np.random.rand(1)[0]
    mask = torch.bernoulli(proba * torch.ones(input_index.shape,
                                                        device=input_index.device))
else:
    mask = torch.bernoulli(args.pkeep * torch.ones(input_index.shape,
                                                        device=input_index.device))
mask = mask.round().to(dtype=torch.int64)
r_indices = torch.randint_like(input_index, args.nb_code)
a_indices = mask*input_index+(1-mask)*r_indices

cls_pred = trans_encoder(a_indices, feat_clip_text)
cls_pred = cls_pred.contiguous()

In [10]:
args.if_maxtest = False

right_num = 0
loss_ce = torch.nn.CrossEntropyLoss()
loss_cls = 0.0
for i in range(bs):
    # loss function     (26), (26, 513)
    loss_cls += loss_ce(cls_pred[i][:m_tokens_len[i] + 1], target[i][:m_tokens_len[i] + 1]) / bs

    # Accuracy
    probs = torch.softmax(cls_pred[i][:m_tokens_len[i] + 1], dim=-1)

    if args.if_maxtest:
        _, cls_pred_index = torch.max(probs, dim=-1)
    else:
        dist = Categorical(probs)
        cls_pred_index = dist.sample()
    right_num += (cls_pred_index.flatten(0) == target[i][:m_tokens_len[i] + 1].flatten(0)).sum().item()

### Eval

In [11]:
val_loader = dataset_TM_eval.DATALoader(args.dataname, False, 32, w_vectorizer)

100%|██████████| 300/300 [00:00<00:00, 2423.09it/s]

Pointer Pointing at 0


In [12]:
class LoggerWriterMock:
    def __init__(self):
        self.info
    def info(self, *args):
        print(*args)
    def add_scalar(self, *args):
        print(*args)
    def add_video(self, *args):
        print(*args)
logger = LoggerWriterMock()
logger.info('test')
writer = LoggerWriterMock()
writer.add_scalar('./Test/FID', 'fid', 'nb_iter')

test
./Test/FID fid nb_iter


In [19]:
args.out_dir = './'
best_fid, best_iter, best_div, best_top1, best_top2, best_top3, best_matching, writer, logger = \
    eval_trans.evaluation_transformer(args.out_dir, 
        val_loader, 
        net, 
        trans_encoder, 
        logger, 
        writer, 
        0, 
        best_fid=1000, 
        best_iter=0, 
        best_div=100, 
        best_top1=0, 
        best_top2=0, 
        best_top3=0, 
        best_matching=100, 
        clip_model=clip_model, 
        eval_wrapper=eval_wrapper)

pose: torch.Size([32, 196, 251])
index_motion: torch.Size([1, 50])
pred_pose: torch.Size([1, 200, 251])
pred_pose_eval: torch.Size([32, 196, 251])


RuntimeError: There were no tensor arguments to this function (e.g., you passed an empty list of Tensors), but no fallback function is registered for schema aten::_cat.  This usually means that this function requires a non-empty list of Tensors.  Available functions are [CPU, CUDA, QuantizedCPU, BackendSelect, Named, AutogradOther, AutogradCPU, AutogradCUDA, AutogradXLA, AutogradNestedTensor, UNKNOWN_TENSOR_TYPE_ID, AutogradPrivateUse1, AutogradPrivateUse2, AutogradPrivateUse3, Tracer, Autocast, Batched, VmapMode].

CPU: registered at /opt/conda/conda-bld/pytorch_1616554786078/work/build/aten/src/ATen/RegisterCPU.cpp:5925 [kernel]
CUDA: registered at /opt/conda/conda-bld/pytorch_1616554786078/work/build/aten/src/ATen/RegisterCUDA.cpp:7100 [kernel]
QuantizedCPU: registered at /opt/conda/conda-bld/pytorch_1616554786078/work/build/aten/src/ATen/RegisterQuantizedCPU.cpp:641 [kernel]
BackendSelect: fallthrough registered at /opt/conda/conda-bld/pytorch_1616554786078/work/aten/src/ATen/core/